In [3]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ SEED 설정 (재현성 확보)
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ Residual Block 정의
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        return nn.ReLU()(x)

# ✅ ASPP Block 정의
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        global_avg = self.conv1_for_global(global_avg)
        global_avg = nn.functional.interpolate(global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True)

        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        return self.conv1x1_out(out)

# ✅ Hybrid Model (EfficientNet + VGG + Residual + ASPP)
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()

        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        self.res_block = ResidualBlock(2048, 2048)  
        self.aspp = ASPP(in_channels=2048, out_channels=256)

        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),  # ✅ Batch Normalization 추가
            nn.Dropout(0.5),  
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = self.eff_conv1x1(eff_features)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = self.vgg_conv1x1(vgg_features)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)  
        fused_features = self.res_block(fused_features)  
        aspp_features = self.aspp(fused_features)

        aspp_features = aspp_features.view(aspp_features.size(0), -1)
        return self.classifier(aspp_features)



# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)

# ✅ EfficientNet 블록 5까지 학습 가능하도록 수정
for name, param in model.efficientnet.named_parameters():
    if "blocks.0" in name or "blocks.1" in name or "blocks.2" in name or "blocks.3" in name or "blocks.4" in name or "blocks.5" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")



Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 1.6662 | Train Acc: 0.3192 | Val Loss: 5.1056 | Val Acc: 0.2034
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 1.4998 | Train Acc: 0.4350 | Val Loss: 337205.5137 | Val Acc: 0.0904

[Epoch 3/100]


Train Loss: 1.4543 | Train Acc: 0.4379 | Val Loss: 7037.0665 | Val Acc: 0.3616
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 1.5463 | Train Acc: 0.3969 | Val Loss: 2.6797 | Val Acc: 0.1977

[Epoch 5/100]


...

Train Loss: 1.2720 | Train Acc: 0.5438 | Val Loss: 805.3684 | Val Acc: 0.3955

[Epoch 15/100]


Train Loss: 1.2057 | Train Acc: 0.5946 | Val Loss: 18191.2802 | Val Acc: 0.2881

[Epoch 16/100]


Train Loss: 1.3349 | Train Acc: 0.5381 | Val Loss: 1.4519 | Val Acc: 0.4463

[Epoch 17/100]


Train Loss: 1.3098 | Train Acc: 0.5226 | Val Loss: 1.4242 | Val Acc: 0.4689

[Epoch 18/100]


Train Loss: 1.2698 | Train Acc: 0.5607 | Val Loss: 1.5520 | Val Acc: 0.3785
Early Stopping Triggered!

Best Accuracy: 0.5763
